# Customer Churn Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark  =SparkSession.builder.appName('customer churn').getOrCreate()

In [12]:
df = spark.read.csv("customer_churn.csv",header=True,inferSchema=True)

In [13]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [22]:
df.describe()

df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [20]:
from pyspark.ml.feature import VectorAssembler 

In [26]:
 assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years','Num_Sites'], outputCol='features')

In [28]:
# now add it to df

In [29]:
output = assembler.transform(df)

In [30]:
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

In [31]:
# we have to focus on features and churn

In [32]:
final_df = output.select('features','churn')

In [33]:
train_df, test_df = final_df.randomSplit([0.7,0.3])

In [37]:
train_df.count()

629

In [38]:
test_df.count()

# Applying Model

271

In [40]:
from pyspark.ml.classification import LogisticRegression

In [45]:
lr = LogisticRegression(labelCol='churn')

In [51]:
 lr_model = lr.fit(train_df) # .transform(train_df)

In [55]:
lr_model.summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.11694681371670...|[0.98396705603635...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[4.03350482507937...|[0.98259611698899...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.19499191651926...|[0.54859410680196...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[5.71562396228277...|[0.99671672094189...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[4.94928540951276...|[0.99296142030378...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[7.14208382351195...|[0.99920952348643...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.13269886790348...|[0.75633662163893...|       0.0|
|[28.0,11204.23,0....|  0.0|[1.42544960213323...|[0.80619132077540...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[3.56064135002826...|[0.97236481692352...|       0.0|
|[29.0,8688.17,1

In [56]:
# lets evaluate it

In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [60]:
prediction = lr_model.evaluate(test_df)

In [63]:
prediction.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11128.95,1....|    0|[3.84086400074062...|[0.97897644282982...|       0.0|
|[28.0,11245.38,0....|    0|[3.24973281279524...|[0.96266351047321...|       0.0|
|[29.0,13240.01,1....|    0|[6.47307483244728...|[0.99845791226784...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.51371630747019...|[0.99598508580560...|       0.0|
|[30.0,8874.83,0.0...|    0|[2.65929552236539...|[0.93458160882345...|       0.0|
|[30.0,10183.98,1....|    0|[2.58775833977496...|[0.93006955963760...|       0.0|
|[30.0,11575.37,1....|    1|[3.72278716222038...|[0.97640372198792...|       0.0|
|[30.0,12788.37,0....|    0|[2.18377712362568...|[0.89878320237061...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.08418165753505...|[0.98344187564344...|       0.0|
|[31.0,11297.57,

In [66]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [67]:
auc = churn_eval.evaluate(prediction.predictions)

In [68]:
auc

0.7631917631917632

In [69]:
# predictions on new data

In [70]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [88]:
finalmodel = lr.fit(final_df) #final_df train_df

In [89]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [90]:
test_new_customers = assembler.transform(new_customers)

tr = finalmodel.transform(test_new_customers)

In [91]:
tr

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [95]:
tr.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [ ]:
So where prediction is 1.o